In [2]:
# -*- coding: utf-8
# VOLT DATA LAB (https://www.voltdata.info/)
# Reinaldo Chaves (@paidatocandeira)
# Mostra quais contratos estaduais têm as empresas de deputados
#

In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.support.select import Select
import pandas as pd

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

Verifica os pagamentos feitos na Execução Orçamentária do Governo do Estado de SP <br>
Fonte: https://www.fazenda.sp.gov.br/SigeoLei131/Paginas/ConsultaDespesaAno.aspx?orgao=

In [ ]:
deputados_socios_empresas = pd.read_csv("resultados/empresas_deputados.csv",sep=',',encoding = 'utf-8', 
                                        converters={'cnpj': lambda x: str(x), 
                                                    'cpf': lambda x: str(x), 
                                                    'documento': lambda x: str(x)})

profile = webdriver.FirefoxProfile()
browser = webdriver.Firefox(profile)
browser.implicitly_wait(10)

# Site that is accessed
browser.get('https://www.fazenda.sp.gov.br/SigeoLei131/Paginas/ConsultaDespesaAno.aspx?orgao=')

# List to store the data
pagamentos = []

for num, row in deputados_socios_empresas.iterrows():
    # Variable with code to search
    empresa = (row['cnpj']).strip()
    deputado = (row['nome_urna']).strip()

    # Search for each code in four years
    for vez in [2015, 2016, 2017, 2018]:
        ano = str(vez)

        # Select year
        Select(browser.find_element_by_name('ctl00$ContentPlaceHolder1$ddlAno')).select_by_visible_text(ano)

        # Fill in the code to search
        browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_rblDoc_0"]').click()
        browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_txtCPF"]').send_keys(empresa)
        browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_btnPesquisar"]').click()

        try:
            found = True
            alert = browser.switch_to.alert
            alert.accept()
            found = False
            # Message shows that the code was not found that year
            print("CNPJ " + empresa + " não encontrado no ano " + ano)
            browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_btnVoltar"]').click()
        except NoAlertPresentException:
            pass

        if found:
            results = browser.find_element_by_xpath("//table[@id='ctl00_ContentPlaceHolder1_gdvCredor']//tr[2]")
            cia = results.find_element_by_xpath("td[1]").text
            valor = results.find_element_by_xpath("td[2]").text

            #Message shows that the code was found that year
            print("CNPJ " + empresa + " encontrado no ano " + ano)

            # Fills dictionary with found data
            dicionario = {"cnpj": empresa,
                          "deputado_relacionado": deputado,
                          "ano": ano,
                          "empresa": cia,
                          "valor": valor,
                         }
            pagamentos.append(dicionario)


            # Go back one screen to do another search
            browser.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_btnVoltar"]').click()

# Create the dataframe
df_pagamentos = pd.DataFrame(pagamentos)

In [6]:
df_pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
ano        8 non-null object
cnpj       8 non-null object
empresa    8 non-null object
valor      8 non-null object
dtypes: object(4)
memory usage: 336.0+ bytes


Pagamentos encontrados

In [7]:
df_pagamentos.reset_index()

,index,ano,cnpj,empresa,valor
0,0,2015,07022473000139,07022473000139 - RAPIDO FENIX VIACAO LTDA,"1.275,90"
1,1,2016,07022473000139,07022473000139 - RAPIDO FENIX VIACAO LTDA,"1.027,50"
2,2,2017,07022473000139,07022473000139 - RAPIDO FENIX VIACAO LTDA,"1.125,36"
3,3,2018,07022473000139,07022473000139 - RAPIDO FENIX VIACAO LTDA,"3.742,70"
4,4,2015,54158522000145,54158522000145 - COMAM CONSORCIO DE MUNICIPIOS...,"462.300,00"
5,5,2016,54158522000145,54158522000145 - COMAM CONSORCIO DE MUNICIPIOS...,"924.600,00"
6,6,2015,61923397000188,61923397000188 - CIMCAL COM.SERV.E SOLUCOES LO...,"1.212,50"
7,7,2016,08095624000141,08095624000141 - BONANZANET INFORM AVAL BENS E...,"1.390,00"


In [4]:
df_pagamentos.to_csv('resultados/pagtos_estaduais_empresas_deputados.csv', index=False, decimal=',')

In [4]:
df_pagamentos = pd.read_csv("resultados/pagtos_estaduais_empresas_deputados.csv",sep=',',encoding = 'utf-8', 
                                        converters={'cnpj': lambda x: str(x)})

In [9]:
def limpeza3(linha):
    valor = linha["valor"]
    valor = valor.replace('.', '')
    valor = valor.replace(',', '.')
    return valor

In [10]:
df_pagamentos['valor'] = df_pagamentos.apply(limpeza3, axis=1)
df_pagamentos['valor'] = df_pagamentos.valor.astype('float64')

In [11]:
"${:,.2f}".format(df_pagamentos.valor.sum())

'$1,396,673.96'

In [8]:
deputados_socios_empresas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547 entries, 0 to 546
Data columns (total 19 columns):
codigo_unico       547 non-null int64
nome_deputado      547 non-null object
uf                 547 non-null object
nome_completo      547 non-null object
sequencial         547 non-null int64
cpf                547 non-null object
nome_urna          547 non-null object
partido_eleicao    547 non-null object
situacao           547 non-null object
Unnamed: 0_x       547 non-null int64
cnpj               547 non-null object
indicador          547 non-null int64
documento          547 non-null object
qualificacao       547 non-null int64
socio              547 non-null object
estado_x           547 non-null object
Unnamed: 0_y       547 non-null int64
nome_empresa       547 non-null object
estado_y           547 non-null object
dtypes: int64(6), object(13)
memory usage: 81.3+ KB


In [2]:
deputados_socios_empresas = pd.read_csv("resultados/empresas_deputados.csv",sep=',',encoding = 'utf-8', 
                                        converters={'cnpj': lambda x: str(x), 
                                                    'cpf': lambda x: str(x), 
                                                    'documento': lambda x: str(x)})

In [3]:
deputados_socios_empresas['cnpj_raiz'] = deputados_socios_empresas.cnpj.str[:8]

In [4]:
deputados_socios_empresas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547 entries, 0 to 546
Data columns (total 20 columns):
codigo_unico       547 non-null int64
nome_deputado      547 non-null object
uf                 547 non-null object
nome_completo      547 non-null object
sequencial         547 non-null int64
cpf                547 non-null object
nome_urna          547 non-null object
partido_eleicao    547 non-null object
situacao           547 non-null object
Unnamed: 0_x       547 non-null int64
cnpj               547 non-null object
indicador          547 non-null int64
documento          547 non-null object
qualificacao       547 non-null int64
socio              547 non-null object
estado_x           547 non-null object
Unnamed: 0_y       547 non-null int64
nome_empresa       547 non-null object
estado_y           547 non-null object
cnpj_raiz          547 non-null object
dtypes: int64(6), object(14)
memory usage: 85.5+ KB


# Contratos de serviços terceirizados 

Fonte: http://www.transparencia.sp.gov.br/contratos_terc.html

In [9]:
terceirizados = pd.read_excel('dados/contratos/contratos_terceirizados_set_2018.xls',sheet_name='Plan1')

In [10]:
terceirizados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10797 entries, 0 to 10796
Data columns (total 12 columns):
Nome da Secretaria                   10797 non-null object
Unidade Gestão Orçamentária (UGO)    10797 non-null object
Unidade Gestão Executora (UGE)       10797 non-null object
Número do Contrato                   10796 non-null object
Início                               10797 non-null datetime64[ns]
Fim                                  10797 non-null datetime64[ns]
Valor                                10797 non-null object
CNPJ_Fornecedor                      10797 non-null object
Nome Fornecedor                      10797 non-null object
Objetivo do Contrato                 10797 non-null object
Classificação do Serviço             10797 non-null object
Tipo do Serviço                      10797 non-null object
dtypes: datetime64[ns](2), object(10)
memory usage: 1012.3+ KB


In [11]:
def limpeza1(linha):
    documento = str(linha["CNPJ_Fornecedor"])
    documento = documento.strip()
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    return str(documento)

def limpeza2(linha):
    documento = str(linha["CNPJ_limpo"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"
    
def limpeza3(linha):
    valor = linha["Valor"]
    valor = valor.replace('.', '')
    valor = valor.replace(',', '.')
    return valor

In [12]:
terceirizados["CNPJ_limpo"] = terceirizados.apply(limpeza1, axis=1)
terceirizados['cnpj_raiz'] = terceirizados.apply(limpeza2, axis=1)
terceirizados['Valor'] = terceirizados.apply(limpeza3, axis=1)

terceirizados['Valor'] = terceirizados.Valor.astype('float64')

In [13]:
terceirizados.reset_index().head()

,index,Nome da Secretaria,Unidade Gestão Orçamentária (UGO),Unidade Gestão Executora (UGE),Número do Contrato,Início,Fim,Valor,CNPJ_Fornecedor,Nome Fornecedor,Objetivo do Contrato,Classificação do Serviço,Tipo do Serviço,CNPJ_limpo,cnpj_raiz
0,0,CASA CIVIL,AGÊNCIA METROPOLITANA DA BAIXADA SANTISTA - AGEM,AGÊNCIA METROPOLITANA DA BAIXADA SANTISTA - AGEM,0001/2015,2015-01-05,2018-10-04,628140.00,10.606.395/0001-24,HS LOCADORA DE VEICULOS LTDA ME,TRANSPORTE MEDIANTE A LOCAÇÃO DE VEÍCULOS COM ...,Transporte,Transporte Mediante Locação de Veículos com Co...,10606395000124,10606395
1,1,CASA CIVIL,AGÊNCIA METROPOLITANA DA BAIXADA SANTISTA - AGEM,AGÊNCIA METROPOLITANA DA BAIXADA SANTISTA - AGEM,0001/2017,2017-04-10,2018-07-09,79500.00,15.182.986/0001-35,GLOBALSERVICE SERVICOS TERCEIRIZADOS EIRELI - ME,PRESTAÇÃO DE SERVIÇOS DE COPEIRAGEM COM 02 (DO...,Apoio administrativo / operacional,Apoio administrativo/operacional,15182986000135,15182986
2,2,CASA CIVIL,AGÊNCIA METROPOLITANA DA BAIXADA SANTISTA - AGEM,AGÊNCIA METROPOLITANA DA BAIXADA SANTISTA - AGEM,0002/2016,2016-05-10,2018-11-09,71511.45,20.811.381/0001-04,VIENA SERVIÇOS TERCEIRIZADOS EIRELI,CONTRATAÇÃO DE EMPRESA ESPECIALIZADA PARA PRES...,Limpeza,Limpeza Predial,20811381000104,20811381
3,3,CASA CIVIL,AGÊNCIA METROPOLITANA DA BAIXADA SANTISTA - AGEM,AGÊNCIA METROPOLITANA DA BAIXADA SANTISTA - AGEM,0003/2016,2016-06-01,2018-05-31,291000.00,65.414.476/0001-14,PHONOWAY COMER.E REPRES.DE SIST.LTDA,CONTRATAÇÃO DE EMPRESA ESPECIALIZADA PARA PRES...,Técnicos,Informática,65414476000114,65414476
4,4,CASA CIVIL,AGÊNCIA METROPOLITANA DA BAIXADA SANTISTA - AGEM,AGÊNCIA METROPOLITANA DA BAIXADA SANTISTA - AGEM,0003/2017,2017-06-01,2019-05-31,1310000.00,08.690.626/0001-89,PLAN CONSULTORIA E PESQUISA EM CIêNCIAS SOCIAI...,CONTRATAÇÃO DE SERVIÇOS DE CONSULTORIA PARA A ...,Técnicos,"Auditoria, consultoria e assessoria financeira...",08690626000189,08690626


Cruzamentos com empresas de deputados

In [14]:
terceirizados_deputados = pd.merge(terceirizados, deputados_socios_empresas.drop_duplicates('cnpj_raiz'), left_on='cnpj_raiz', right_on='cnpj_raiz')

In [15]:
terceirizados_deputados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 33 columns):
Nome da Secretaria                   0 non-null object
Unidade Gestão Orçamentária (UGO)    0 non-null object
Unidade Gestão Executora (UGE)       0 non-null object
Número do Contrato                   0 non-null object
Início                               0 non-null datetime64[ns]
Fim                                  0 non-null datetime64[ns]
Valor                                0 non-null float64
CNPJ_Fornecedor                      0 non-null object
Nome Fornecedor                      0 non-null object
Objetivo do Contrato                 0 non-null object
Classificação do Serviço             0 non-null object
Tipo do Serviço                      0 non-null object
CNPJ_limpo                           0 non-null object
cnpj_raiz                            0 non-null object
codigo_unico                         0 non-null int64
nome_deputado                        0 non-null object
uf              